In [7]:
# As you have seen, there are many ways to make a request.
# Here is one:
import requests

some_url = 'https://data-cloud.flightradar24.com/zones/fcgi/feed.js?faa=1&bounds=50.783%2C47.254%2C0.274%2C21.366&satellite=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=14400&gliders=1&stats=1&enc=09ze1OJdYLT0I3QsCeHFDbe9-5E1Ltd51l4JiiQx0SU'
payload = requests.get(some_url, headers={'User-Agent': 'Mozilla/5.0'})

In [8]:
payload.text

'{"full_count":18709,"version":4,"30afaa2c":["",50.604,6.447,292,45000,488,"3563","F-LFSN1","GA6C","",1686585798,"","","",0,-64,"GA6C",0,""]\n,"30afc3c8":["",48.601,3.207,332,43000,453,"4024","F-LFPC1","GL5T","",1686585800,"","","",0,-64,"GL5T",0,""]\n,"30af87d0":["4D2480",48.207,5.666,304,43000,448,"7523","F-LFLI2","CL35","9H-VCU",1686585800,"ATH","FAB","VJT473",0,-64,"VJT473",0,""]\n,"30af9b04":["A7A28D",50.252,4.379,318,43000,403,"6756","T-EHGR86","C68A","N591QS",1686585797,"NCE","NHT","",0,0,"N591QS",0,"EJA"]\n,"30affc13":["493294",50.525,3.871,114,41000,440,"2164","F-ETIE1","C68A","CS-LTT",1686585798,"BQH","RMI","",0,0,"NJE3UB",0,"NJE"]\n,"30afa5fb":["",49.326,2.943,213,41000,402,"1000","F-LFAT2","PC24","",1686585800,"","","",0,0,"PC24",0,""]\n,"30afee94":["4A08E9",47.562,10.981,112,41000,472,"2127","F-EDMO1","B737","YR-BGI",1686585799,"CDG","OTP","RO384",0,0,"ROT384J",0,"ROT"]\n,"30afd902":["710415",47.718,7.693,120,40975,502,"2726","F-LSGG3","B789","HZ-AR23",1686585799,"LGW","JE

In [12]:
# Here anotherone
from urllib.request import Request, urlopen
import datetime
import json

import pandas as pd

In [13]:
# To obtain this URL, you need to inspect using F12 and go to the network tab. 
# If you wait for 5 seconds, you should copy the last row that appears.

url = "https://data-cloud.flightradar24.com/zones/fcgi/feed.js?faa=1&bounds=52.418%2C45.438%2C4.613%2C17.027&satellite=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=14400&gliders=1&stats=1"

In [16]:
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
data = urlopen(req, timeout=10).read()

print(data)
print()
print(type(data).__name__)

b'{"full_count":12457,"version":4,"30b205b0":["4D02D9",46.934,5.977,324,43000,403,"","F-LSZC1","PC24","LX-PCI",1686640463,"PEG","MME","",0,0,"JFA18V",0,"JFA"]\n,"30b11242":["3C670B",51.257,7.867,147,41000,506,"","F-EDLE3","A359","D-AIXK",1686640465,"DEN","MUC","LH481",0,0,"DLH481",0,"DLH"]\n,"30b215e6":["406D70",51.895,4.800,114,40975,496,"","F-EDLN2","B788","G-TUII",1686640463,"MAN","AYT","BY512",0,-64,"TOM9VK",0,"TUI"]\n,"30b2144f":["49D39B",50.004,8.637,260,40000,413,"","F-EDRY1","BE40","OK-BII",1686640465,"PRG","OVD","",0,0,"JBC511I",0,"JBC"]\n,"30b1d3b6":["405BFE",46.520,16.117,311,40000,452,"","F-LJMB1","B772","G-YMMU",1686640464,"TLV","LHR","BA166",0,0,"BAW166E",0,"BAW"]\n,"30b21930":["49328E",49.502,9.396,5,40000,419,"","F-EDQG1","C68A","CS-LTN",1686640464,"LUG","RKE","",0,0,"NJE022W",0,"NJE"]\n,"30b1f313":["4B8E41",47.994,16.459,292,40000,446,"","F-LHSM3","B738","TC-CRA",1686640464,"SAW","MAN","PC1181",0,0,"PGT1181",0,"PGT"]\n,"30b1e826":["4CADF2",52.390,6.037,54,39025,443,"",

In [35]:
def extractFlights(_data):
    """
    Extracts flight data from the provided JSON data and returns a list of flight dictionaries.

    Args:
        _data (bytes): The JSON data as bytes.

    Returns:
        list: A list of flight dictionaries containing flight information.
    """

    result = []
    
    raw_json = _data.decode('utf8').replace("'", '"')
    temp_json = json.loads(raw_json)

    # print(json.dumps(temp_json, indent=4))
    # print()

    count = temp_json['full_count']
    stats = temp_json['stats']

    # print('full_count =', count)
    # print('Stats type=', type(stats))
    # print()
    
    del temp_json['full_count']
    del temp_json['version']
    del temp_json['stats']

    # print('New JSON:', json.dumps(temp_json, indent=4))

    lista = list(temp_json.keys())
    
    # print(lista)

    raw_flights = []
    for leg in lista:
        raw_leg = temp_json[leg]
        raw_leg.append(leg)
        raw_flights.append(raw_leg)
        # print('raw_leg =', raw_leg)
        # print()
    
    # print('raw_flights =', raw_flights[0])

    for flight in raw_flights:
    
        template = {
            "snapID" : str(datetime.datetime.now().strftime("%Y%m%d%H%M")) + flight[19],
            "leg": flight[19],
            "id": flight[0],
            "latitude": flight[1],
            "longitude": flight[2],
            "track": flight[3],
            "altitude": flight[4],
            "speed": flight[5],
            "ot0": flight[6],
            "airSpace": flight[7],
            "acType": flight[8],
            "acPlate": flight[9],
            "snapshot": flight[10],
            "airportDep": flight[11],
            "airportArr": flight[12],
            "flightNumber": flight[13],
            "callsign": flight[16],
            "airlineOaci": flight[18],
            "airlineCode": flight[13][:2]
        }
        result.append(template)

    return result

In [36]:
raw = extractFlights(data)

{"full_count":12457,"version":4,"30b205b0":["4D02D9",46.934,5.977,324,43000,403,"","F-LSZC1","PC24","LX-PCI",1686640463,"PEG","MME","",0,0,"JFA18V",0,"JFA"]
,"30b11242":["3C670B",51.257,7.867,147,41000,506,"","F-EDLE3","A359","D-AIXK",1686640465,"DEN","MUC","LH481",0,0,"DLH481",0,"DLH"]
,"30b215e6":["406D70",51.895,4.800,114,40975,496,"","F-EDLN2","B788","G-TUII",1686640463,"MAN","AYT","BY512",0,-64,"TOM9VK",0,"TUI"]
,"30b2144f":["49D39B",50.004,8.637,260,40000,413,"","F-EDRY1","BE40","OK-BII",1686640465,"PRG","OVD","",0,0,"JBC511I",0,"JBC"]
,"30b1d3b6":["405BFE",46.520,16.117,311,40000,452,"","F-LJMB1","B772","G-YMMU",1686640464,"TLV","LHR","BA166",0,0,"BAW166E",0,"BAW"]
,"30b21930":["49328E",49.502,9.396,5,40000,419,"","F-EDQG1","C68A","CS-LTN",1686640464,"LUG","RKE","",0,0,"NJE022W",0,"NJE"]
,"30b1f313":["4B8E41",47.994,16.459,292,40000,446,"","F-LHSM3","B738","TC-CRA",1686640464,"SAW","MAN","PC1181",0,0,"PGT1181",0,"PGT"]
,"30b1e826":["4CADF2",52.390,6.037,54,39025,443,"","F-EDVM1"

In [34]:
result = pd.DataFrame(raw, columns=raw[0].keys())
result

,snapID,leg,id,latitude,longitude,track,altitude,speed,ot0,airSpace,acType,acPlate,snapshot,airportDep,airportArr,flightNumber,callsign,airlineOaci,airlineCode
0,20230613093230b205b0,30b205b0,4D02D9,46.934,5.977,324,43000,403,,F-LSZC1,PC24,LX-PCI,1686640463,PEG,MME,,JFA18V,JFA,
1,20230613093230b11242,30b11242,3C670B,51.257,7.867,147,41000,506,,F-EDLE3,A359,D-AIXK,1686640465,DEN,MUC,LH481,DLH481,DLH,LH
2,20230613093230b215e6,30b215e6,406D70,51.895,4.800,114,40975,496,,F-EDLN2,B788,G-TUII,1686640463,MAN,AYT,BY512,TOM9VK,TUI,BY
3,20230613093230b2144f,30b2144f,49D39B,50.004,8.637,260,40000,413,,F-EDRY1,BE40,OK-BII,1686640465,PRG,OVD,,JBC511I,JBC,
4,20230613093230b1d3b6,30b1d3b6,405BFE,46.520,16.117,311,40000,452,,F-LJMB1,B772,G-YMMU,1686640464,TLV,LHR,BA166,BAW166E,BAW,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,20230613093230b20f11,30b20f11,3C3E10,50.863,7.131,123,0,4,,F-EDDK3,GRND,,1686640464,CGN,,MT42,MT42,,MT
853,20230613093230b211ae,30b211ae,440822,48.118,16.573,205,0,7,,F-LOWW8,E195,OE-LWF,1686640445,MUC,VIE,OS112,AUA2MU,AUA,OS
854,20230613093230b21290,30b21290,49F00F,50.093,14.282,350,0,0,,F-LKPR10,GRND,,1686640464,,,,UDRZBA2,,
855,20230613093230b21041,30b21041,3C6754,50.040,8.576,70,0,26,,F-EDDF6,A320,D-AIZT,1686640465,ZRH,FRA,LH1183,DLH3C,DLH,LH
